In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
import pandas as pd

## Sobel

In [ ]:
def convolve2d(image, kernel):
    kernel_height, kernel_width = kernel.shape
    # Calculate the padding size
    pad_height = kernel_height // 2
    pad_width = kernel_width // 2
    
    padded_image = np.pad(image, ((pad_height, pad_height), (pad_width, pad_width)), mode='constant', constant_values=0)
    padded_height, padded_width = padded_image.shape
    output = np.zeros_like(image, dtype=np.float32)

    # Perform convolution
    for i in range(pad_height, padded_height - pad_height):
        for j in range(pad_width, padded_width - pad_width):
            region = padded_image[i - pad_height:i + pad_height + 1, j - pad_width:j + pad_width + 1]
            output[i - pad_height, j - pad_width] = np.sum(region * kernel)

    return output


def apply_sobel_filters(image):
    # Sobel kernels
    sobel_x = np.array([[1, 0, -1],
                        [2, 0, -2],
                        [1, 0, -1]])

    sobel_y = np.array([[1, 2, 1],
                        [0, 0, 0],
                        [-1, -2, -1]])

    gradient_x = convolve2d(image, sobel_x)
    gradient_y = convolve2d(image, sobel_y)
    gradient_magnitude = np.sqrt(gradient_x ** 2 + gradient_y ** 2)

    return gradient_magnitude, gradient_x, gradient_y

## Gaussian filter

In [ ]:
def gaussian_kernel(size, sigma=1):
    """Generates a Gaussian kernel."""
    kernel = np.fromfunction(
        lambda x, y: (1 / (2 * np.pi * sigma ** 2)) *
                     np.exp(-((x - (size - 1) / 2) ** 2 + (y - (size - 1) / 2) ** 2) / (2 * sigma ** 2)),
        (size, size)
    )
    return kernel / np.sum(kernel)  # Normalize the kernel


def apply_gaussian_filter(image, kernel_size=5, sigma=1):
    kernel = gaussian_kernel(kernel_size, sigma)
    filtered_image = convolve2d(image, kernel)
    return filtered_image


## HOG

In [ ]:
from skimage.feature import hog
from skimage import exposure

def apply_hog(image):
    fd, hog_image = hog(
        image,
        orientations=2,
        pixels_per_cell=(4, 4),
        cells_per_block=(1, 1),
        block_norm='L2-Hys',
        visualize=True
    )

    # Normalize the HOG image for better visualization
    hog_image = exposure.rescale_intensity(hog_image, in_range=(0, 10))

    return fd, hog_image  # Return both the HOG image and the feature descriptor

In [ ]:
# Fetch the MNIST dataset
mnist = fetch_openml('mnist_784', version=1)
df = pd.DataFrame(mnist.data)
df['label'] = mnist.target  # Add the labels to the DataFrame
print(df.head())
df.to_csv('raw.csv', index=False)
print("MNIST dataset has been saved to raw.csv")

# Extract images and labels
images = df.iloc[:, :-1].values  # All columns except the last
labels = df.iloc[:, -1].values  # Last column (labels)

sobel_images = []
hog_images = []
hog_fds = []
gaussian_images = []

# Process each image
counter = 0
for image in images:
    image_reshaped = image.reshape(28, 28)
    gaussian_image = apply_gaussian_filter(image_reshaped, kernel_size=5, sigma=1)
    gaussian_images.append(gaussian_image.flatten())
    print("Gau")
    sobel_image = apply_sobel_filters(image_reshaped)
    sobel_images.append(sobel_image.flatten())
    print("Sob")
    fd, hog_image = apply_hog(image)
    hog_images.append(hog_image.flatten())
    hog_fds.append(fd)
    print("HOG")
    print(counter)
    counter += 1

sobel_df = pd.DataFrame(sobel_images)
gaussian_df = pd.DataFrame(gaussian_images)
hog_df = pd.DataFrame(hog_fds)
sobel_hog_df = df.concat([sobel_df, hog_df], axis=1)
gaussian_hog_df = df.concat([gaussian_df, hog_df], axis=1)

sobel_df['label'] = labels
sobel_df.to_csv('sobel.csv', index=False)
print("sobel images have been saved to sobel.csv")

gaussian_df['label'] = labels
gaussian_df.to_csv('gaussian.csv', index=False)
print("gaussian images have been saved to gaussian.csv")

sobel_hog_df['label'] = labels
sobel_hog_df.to_csv('sobel_hog.csv', index=False)
print("sobel+hog features have been saved to sobel_hog.csv")

gaussian_hog_df['label'] = labels
gaussian_hog_df.to_csv('gaussian_hog.csv', index=False)
print("gaussian+hog features have been saved to gaussian_hog.csv")